# Project Cryosphere by Matthias Göbel and Philipp Gregor
## Part 1

In [ ]:
# imports and defaults
import pandas as pd  
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_rows = 14
import seaborn as sns
sns.set_style('ticks')
sns.set_context('talk')
import scipy.constants as const

In [ ]:
# read the data
df = pd.read_csv('data/aws_data_zhadang_localtime.csv', index_col=0, parse_dates=True)

## 1 Radiative Fluxes

In this first part, we look at the different longwave and shortwave contributions to the net radiative flux:
$ 
Q_{\mathrm{net}}=\mathrm{SW_{in}+SW_{out}+LW_{in}+LW_{out}}
$

### 1.1 Accuracy of the different sensors

As described in *Maussion et al. (2011)* three types of sensors (all from Campbell) were used to measure the radiation fluxes:
- NR-Lite for net radiation (http://s.campbellsci.com/documents/us/manuals/nr-lite.pdf)
- CS300 for incoming and outgoing shortwave radiation (https://s.campbellsci.com/documents/us/manuals/cs300.pdf)
- IRTS-P for outgoing longwave radiation (http://s.campbellsci.com/documents/us/manuals/irts.pdf)

The IRTS-P measures the outgoing longwave radiation and computes the surface temperature $T$ from it. We use Stefan-Boltzmann's law to recalculate the flux $F$:

$F=\sigma T^4$

where $\sigma$ is the Stefan-Boltzmann constant.


In [ ]:
df["LWOUT"]=-const.sigma*(df.SURFTEMP+273.15)**4  #convert surface temperature to Kelvin and calculate LWOUT with Stefan-Boltzmann

In the manuals we can find the accuracy of the three instruments. The absolute error of the IRTS-P is given in $^\circ \mathrm{C}$: $\Delta T=\pm 0.3 \,^\circ \mathrm{C}$. We can calculate the relative error of outgoing longwave radiation $F$ by using Stefan-Boltzmann's law and the variance formula for the propagation of uncertainty (https://en.wikipedia.org/wiki/Propagation_of_uncertainty):
$$\Delta F=\left|\frac{\partial F}{\partial T} \Delta T \right|=4F\frac{\Delta T}{T}$$
$$\Rightarrow \frac{\Delta F}{F}=4\frac{\Delta T}{T}$$
To estimate this expression we use the mean value of the surface temperature:


In [ ]:
dF=4*0.3/(df.SURFTEMP+273).mean()*100 #relative error in %
dF

The relative error is then only $\Delta F =\pm 0.5\%$. This is  however underestimated since the calculation of $F$ with Stefan-Boltzmann's law is not exact.

Accuracy of the instruments:
- NR-Lite:  
    - Directional error: $< 30 \,\mathrm{W m^{-2}}$(at an incidence angle of 0 - 60° and radiation $1000 \,\mathrm{W m^{-2}}$)
    - Sensor asymmetry: $\pm 5\%$ 
- CS300: $\pm 5\%$ 
- IRTS: >$\pm 0.5 \%$ 

### 1.2 Spectral ranges
In the manuals we also find the respective spectral ranges of the used instruments:
- NR-Lite: $\mathrm{0.2 - 100 \,\mu m}$
- CS300: $\mathrm{0.36 - 1.12 \,\mu m}$
- IRTS-P: $\mathrm{6 - 14 \,\mu m}$


Since the AWS was not equipped with a sensor measuring incoming longwave radiation, it has to be computed as a residual of the radiation budget equation. However, since the NR-Lite can measure lower wavelengths than the CS300 and higher wavelengths than the IRTS-P $\mathrm{SW_{in},\,SW_{out}\, and\,LW_{out}}$ will be underestimated. Thus, $\mathrm{LW_{in}}$ will be overestimated.

### 1.3 Analysing the different contributions

In [ ]:
# rad=np.pi/180
# S0=1367
# eps = 23.4398*rad
# w= 282.895*rad
# e = 0.016704
# phi=47*rad
# delta=-23.44*(np.cos(2*np.pi/365*(np.arange(274,274+700)+10)))*rad
# th=np.arcsin(delta/eps)
# h0=np.arccos(-np.tan(phi)*np.tan(delta))
# Qd=S0/np.pi*(1+e*np.cos(th-w))**2*(h0*np.sin(phi)*np.sin(delta)+np.cos(phi)*np.cos(delta)*np.sin(h0))
# plt.plot(Qd)

Having in mind that the spectral ranges of the different instruments are not matching, we now compute $\mathrm{LW_{in}}$ as the residual of the other fluxes and then plot the monthly averages of the different components.

In [ ]:
df.SWOUT=-df.SWOUT #correct sign 

In [ ]:
df["LWIN"]=df.NETRAD-df.SWIN-df.SWOUT-df.LWOUT #calculate LWIN as residual

Below we plot the monthly averages of the different components. We take the absolute values (except for net radiation) so that we can better compare the fluxes. 

In [ ]:
#To visualize the difference between individual days in one month we use the monthly standard deviation of daily average values as errorbars. In this way we disregard the diurnal cycle.

In [ ]:
df_monthly=df.resample('MS').mean()
#df_monthly_std=(df.resample('D').mean()).resample('MS').std()
for i in [["NETRAD","Net radiation"],
          ["SWIN","Incoming shortwave radiation"],["SWOUT","Outgoing shortwave radiation"],
          ["LWIN","Incoming longwave radiation"],["LWOUT","Outgoing longwave radiation"]]:
    plt.figure()
    d=df_monthly[i[0]]
    
    if i[0]!="NETRAD":
        d=abs(d) #use absolute values
       # plt.ylim(0,400)

    d.plot(marker="o",linestyle="")#,yerr=df_monthly_std[i[0]])
    plt.xlabel("")
    plt.ylabel("Radiative flux [$\mathrm{Wm^{-2}}$]")
    plt.xlim("2010-09-01","2012-09-01")
    plt.suptitle(i[1])
    plt.grid()


In winter, from November to February, negative net radiation prevails, whereas in summer, from May to September, positive values prevail. In the transition months March, April and October both, positive and negative values occur almost equally often.

The other components and their respective driving processes will be addressed in the following section.

### 1.4 Driving processes

**Daily averages plot**

In [ ]:
df_daily=df.resample('D').mean().copy()
for i in [["NETRAD","Net radiation"],
          ["SWIN","Incoming shortwave radiation"],["SWOUT","Outgoing shortwave radiation"],
          ["LWIN","Incoming longwave radiation"],["LWOUT","Outgoing longwave radiation"]]:
    plt.figure()
    d=df_daily[i[0]]
    
    if i[0]!="NETRAD":
        d=abs(d) #use absolute values
       # plt.ylim(0,400)
  #  if i[0]!="SWIN":
   #     continue
    d.plot(marker=".",linestyle="")
   # plt.scatter(d.index,d.values,c=np.linspace(0,1,len(d)),cmap="inferno")
    plt.xlabel("")
    plt.ylabel("Radiative flux [$\mathrm{Wm^{-2}}$]")
    plt.xlim("2010-09-01","2012-10-01")
    plt.suptitle(i[1])
    plt.grid()
    


#### Incoming shortwave radiation
Incoming shortwave radiation mainly follows the seasonal cycle of solar irradiance at the top of the atmosphere: Minimum in winter, maximum in summer. Direct and diffuse solar radiation is the main driver. The solar irradiance at the TOA minus the radiation that is reflected or absorbed in the clear atmosphere gives the upper limit for the incoming shortwave radiation. The daily average plot above shows this quite well.

If clouds are present, incoming shortwave radiation is reduced. Especially in July, August and September solar irradiance is far below the upper limit on many days, as can be seen on the daily averages plot, again. Therefore the largest fluxes occur in May. 

Relative humidity can serve as a proxy for cloud cover. Below we plot incoming shortwave radiation versus relative humidity for the month of July (2011 and 2012). We take only one month to minimize the effect of the seasonal cycle. This leads to a correlation coefficient of -0.64. The higher relative humidity, the lower the incoming shortwave radiation.

The last and probably least important contribution is radiation that is reflected from the ground and then reflected again from clouds and radiation that is reflected from surrounding surfaces (ice and snow) and reaches the sensor from above. In this way incoming shortwave radiation can exceed the mentioned upper limit.

In [ ]:
df_mon=df_daily.loc[df_daily.index.month==7]
d1=df_mon["RH"]
d2=df_mon["SWIN"]
plt.scatter(d1,d2)
plt.ylabel("Incoming shortwave radiation [$\mathrm{Wm^{-2}}$]")
plt.xlabel("Relative humidity [%]")
corr=np.corrcoef(d1,d2)[0,1]
plt.text(min(d1),min(d2),"Correlation coefficient: {0}".format(np.round(corr,2)))
plt.suptitle("Correlation of $\mathrm{SW_{in}}$ and relative humidity in July");

#### Outgoing shortwave radiation
Outgoing shortwave radiation is the incoming shortwave radiation that is reflected from the surface. It depends on the value of the incoming shortwave radiation (and thus on seasonal cycle and clouds) and the albedo of the surface. 
To investigate the latter effect we plot the daily ratio of outgoing to incoming shortwave radiation below.

Since snow has a higher albedo than ice, this ratio is usually higher in winter than in summer. The fresher the snow, the higher the albedo is. To see the effect of snowfall, we plot vertical lines for days with a height increase of more than 1 mm. In the plot we can see that also in summer there is snowfall where the ratio  $\mathrm{SW_{out}/SW_{in}}$ increases rapidly.

<span class="burk">BUT ALSO INCREASE WHERE NO SNOWFALL??</span>

In [ ]:
df_daily["SNOW"]=0
df_daily["SNOW"][1:].loc[df_daily["SR50"][1:]>df_daily["SR50"][:-1]+0.0001]=1

In [ ]:
(-df_daily["SWOUT"]/df_daily["SWIN"]).plot()
plt.ylabel("")
plt.xlabel("")
plt.suptitle("Daily albedo: $\mathrm{SW_{out}/SW_{in}}$ and snowfall events (green vertical lines)");
plt.ylim(0,1)

for t in df_daily.index[df_daily["SNOW"]==1]:
    plt.axvline(x=t,linewidth=1,color="g")


#### Incoming longwave radiation
The source of incoming longwave radiation is thermal radiation from the atmosphere and clouds. As before for the shortwave radiation we can easily see the effect of clouds in a scatter plot of incoming longwave radiation versus relative humidity. In August we get a correlation coefficient of 0.8.

The variability, however, is lower than for the shortwave fluxes, as we can see in the daily averages plot.
$\mathrm{LW_{out}}$ also shows a seasonal cycle, since the warmer atmosphere/clouds in summer radiates more energy away.


In [ ]:
df_mon=df_daily.loc[df_daily.index.month==8]
d1=df_mon["RH"]
d2=df_mon["LWIN"]
plt.scatter(d1,d2)
plt.ylabel("Incoming longwave radiation [$\mathrm{Wm^{-2}}$]")
plt.xlabel("Relative humidity [%]")
corr=np.corrcoef(d1,d2)[0,1]
plt.text(min(d1),min(d2),"Correlation coefficient: {0}".format(np.round(corr,2)))
plt.suptitle("Correlation of $\mathrm{LW_{in}}$ and relative humidity in August");

#### Outgoing longwave radiation


The outgoing longwave radiation depends on the temperature and emissivity of the surface.
The temperature dependence leads to the usual seasonal cycle. 

<span class="burk">NOT SURE ABOUT THIS -> SEARCH LITERATURE:</span>
Emissivity depends on the surface characteristics. Snow has a lower emissivity than ice, which is revealed by a scatter plot of $\mathrm{LW_{out}}$ versus $\mathrm{SW_{out}/SW_{in}}$.

In [ ]:
df_mon=df_daily.loc[df_daily.index.month==4]
d1=-df_mon["SWOUT"]/df_mon["SWIN"]
d2=-df_mon["LWOUT"]
plt.scatter(d1,d2)
plt.ylabel("Outgoing longwave radiation [$\mathrm{Wm^{-2}}$]")
plt.xlabel("$\mathrm{SW_{out}/SW_{in}}$")
corr=np.corrcoef(d1,d2)[0,1]
plt.text(min(d1),min(d2),"Correlation coefficient: {0}".format(np.round(corr,2)))
plt.suptitle("Correlation of $\mathrm{LW_{out}}$ and $\mathrm{SW_{out}/SW_{in}}$  in February");

## 2 Turbulent Fluxes

## 3 Residual Flux $F$ and melt energy $Q_M$

## 4 Mass balance

## 5 An attempt of validation

## References


- Maussion, Fabien, et al. "Glaciological field studies at Zhadang Glacier (5500–6095 m), Tibetan Plateau." Workshop on the use of automatic measuring systems on glaciers–Extended abstracts and recommendations of the IASC Workshop, edited by: Tijm-Reijmer, CH and Oerlemans, J. 2011.